# How to have chains

# 1. Create a simple chain

In [9]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
import warnings
from langchain_core.output_parsers import StrOutputParser
from langchain.agents import initialize_agent
from langchain.tools.retriever import create_retriever_tool
from dotenv import load_dotenv, find_dotenv
import os
from langchain.chains import SimpleSequentialChain
from langchain.prompts.prompt import PromptTemplate
from langchain.chains.llm import LLMChain


warnings.filterwarnings("ignore")
load_dotenv(find_dotenv(), override=True)
openai_api_key=os.getenv('OPENAI_API_KEY')


model = "gpt-3.5-turbo"
llm = ChatOpenAI(temperature=0.5, model=model, max_tokens=4096)

In [10]:
from langchain.agents.load_tools import get_all_tool_names, load_tools
print(get_all_tool_names())

['requests', 'requests_get', 'requests_post', 'requests_patch', 'requests_put', 'requests_delete', 'terminal', 'sleep', 'wolfram-alpha', 'google-search', 'google-search-results-json', 'searx-search-results-json', 'bing-search', 'metaphor-search', 'ddg-search', 'google-lens', 'google-serper', 'google-scholar', 'google-finance', 'google-trends', 'google-jobs', 'google-serper-results-json', 'searchapi', 'searchapi-results-json', 'serpapi', 'dalle-image-generator', 'twilio', 'searx-search', 'merriam-webster', 'wikipedia', 'arxiv', 'golden-query', 'pubmed', 'human', 'awslambda', 'stackexchange', 'sceneXplain', 'graphql', 'openweathermap-api', 'dataforseo-api-search', 'dataforseo-api-search-json', 'eleven_labs_text2speech', 'google_cloud_texttospeech', 'reddit_search', 'news-api', 'tmdb-api', 'podcast-api', 'memorize', 'llm-math', 'open-meteo-api']


# Chromadb

In [11]:
import chromadb
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

collection_name = "chatbot"
client = chromadb.Client()
client.heartbeat()
langchain_chroma = Chroma(
    client=client,
    collection_name=collection_name,
    embedding_function=embedding_function,
)

# Loading tools

In [12]:
tools = load_tools(["ddg-search", "llm-math"], llm=llm)

# find the name and description of each tool
print(tools[0].name, tools[0].description)
print(tools[1].name, tools[1].description, "\n\n")

## tool name and description

#('duckduckgo_search',
# 'A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.')

from langchain.agents import initialize_agent
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", input_key="input", output_key="output",return_messages=True) 
agent = initialize_agent(
        agent="zero-shot-react-description",
        tools=tools,
        llm=llm,
        memory=memory,
        return_intermediate_steps= True, # Make sure you set it to True
        verbose=True
    )
print(agent.agent.llm_chain.prompt.template)

duckduckgo_search A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.
Calculator Useful for when you need to answer questions about math. 


Answer the following questions as best you can. You have access to the following tools:

duckduckgo_search: A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.
Calculator: Useful for when you need to answer questions about math.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [duckduckgo_search, Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: 

# Create a chain with an agent
This agent will be used to answer a list of embeddings questions and predefined QA, there will be a tool that will be watching the conversation and will trigger to create an order.
* create a prompt for the agent convesation
* create a retriever tool for the QA
* create a retriever tool for the inventory
* create a tool to identify the order

In [18]:
from langchain import hub
from langchain.agents import AgentExecutor, create_json_chat_agent

retriever = langchain_chroma.as_retriever(search_type="mmr")

retriever_tool = create_retriever_tool(
    retriever,
    "qa_retriever",
    "Buscar en la base de datos de QA",
)
tools = [retriever_tool]
prompt = hub.pull("hwchase17/react-chat-json")

agent_json = create_json_chat_agent(llm, tools, prompt)


agent_executor = AgentExecutor(
    agent=agent_json, tools=tools, verbose=True, handle_parsing_errors=True
)
agent_executor.invoke({"input": "cuales servicios tienes?"})





> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "Ofrezco una variedad de servicios, como responder preguntas, proporcionar información, explicaciones detalladas sobre diversos temas, entre otros. ¿En qué puedo ayudarte hoy?"
}
```

> Finished chain.


{'input': 'cuales servicios tienes?',
 'output': 'Ofrezco una variedad de servicios, como responder preguntas, proporcionar información, explicaciones detalladas sobre diversos temas, entre otros. ¿En qué puedo ayudarte hoy?'}